#  Generate Some Transfered Sentences and get Style Scores

## Imports

In [21]:
%load_ext autoreload
%autoreload 2
    
import sys, os
import numpy as np
sys.path.append('../paraphrase/')
sys.path.append('../jointclassifier/')
from paraphraser_args import ModelArguments as pma, DataTrainingArguments as pda, TrainingArguments as pta
from paraphraser_dataloader import load_dataset as pld, load_dataset_style as lds
from paraphraser_dataloader import load_dataset_pseudo as ldp
from paraphraser_trainer import ParaphraserTrainer
from transformers import AutoConfig, AutoTokenizer, AutoModelWithLMHead, HfArgumentParser
from joint_args import ModelArguments as jma, DataTrainingArguments as jda, TrainingArguments as jta
from joint_dataloader import load_dataset as jld
from joint_trainer import JointTrainer
from joint_model_v1 import JointSeqClassifier

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tqdm.notebook import tqdm, trange
from torch import cuda, no_grad

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'load_dataset_pseudo' from 'paraphraser_dataloader' (../paraphrase/paraphraser_dataloader.py)

## Load in desired dataset and paraphraser model
In the cell below, define the dataset you want to work with and the paraphraser model (here a `"t5-small"` [from Hugging Face](https://huggingface.co/t5-small))

In [18]:
data_dir = '../data/shakespeare/'
#"../data/processed_filtered/"

In [19]:
paraphrase_model_name = "t5_transfer_shakespeare"
paraphrase_task = 'shakespeare'
prompt_task = paraphrase_task + "_prompt"


paraphrase_model_nick = "t5_transfer_shakespeare"
paraphrase_model_type = 't5-small'
output_dir = "../models/"
epochs = "3"
train_batch_size = "16"
eval_batch_size = "16"
save_log_steps = "400"

parser = HfArgumentParser((pma, pda, pta))
model_args_para, data_args_para, training_args_para = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    paraphrase_model_name,
    "--model_nick",
    paraphrase_model_nick,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, paraphrase_model_nick),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    save_log_steps,
    "--save_steps",
    save_log_steps,
    "--data_parallel",
    "True"
])


PyTorch: setting up devices


In [6]:
joint_task = "abstract+shakespeare" #"formality+emo"
data_dir = "../data/processed_filtered/"
joint_model_name = "distilbert-base-uncased"
joint_model_nick = "distilbert_uncased_2"
output_dir = "../models/"
freeze_encoder = "False"
skip_preclassifier = "False"
train_jointly = "True"
epochs = "5"
train_batch_size = "256"
eval_batch_size = "512"
log_save_steps = "200"

parser = HfArgumentParser((jma, jda, jta))
model_args_joint, data_args_joint, training_args_joint = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    joint_model_name,
    "--model_nick",
    joint_model_nick,
    "--task",
    joint_task,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, joint_model_nick, joint_task, 'joint'),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--freeze_encoder",
    freeze_encoder,
    "--skip_preclassifier",
    skip_preclassifier,
    "--train_jointly",
    train_jointly,
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    log_save_steps,
    "--save_steps",
    log_save_steps
])


PyTorch: setting up devices


In [20]:
# Create the paraphraser tokenizer and dataset objects
para_tokenizer = AutoTokenizer.from_pretrained(paraphrase_model_type, cache_dir=model_args_para.cache_dir,
                                         model_max_length = data_args_para.max_seq_len)
dataset = ldp(data_dir, para_tokenizer,
                            task=prompt_task, mode="train", n_proc=6000)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at ../models/cache/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
  

FileNotFoundError: [Errno 2] No such file or directory: '../data/shakespeare/shakespeare_prompt/config.json'

In [8]:
# Use the paraphrase configuration defined above to create the model
model = AutoModelWithLMHead.from_pretrained(os.path.join(output_dir, paraphrase_model_name))
#training_args_para.output_dir)

/home/dmac/Documents/MIMS Coursework/Capstone/env_marvin/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:966: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file ../models/t5_transfer_shakespeare/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0

## Use the Paraphraser to Generate Predictions

In [9]:
sampler = SequentialSampler(dataset)
dataloader = DataLoader(dataset, sampler=sampler, batch_size=16)

num_return_sequences = 3

device = ("cuda" if cuda.is_available() else "cpu") #and not self.args.no_cuda
model = model.to(device)
model.eval()
predicted1 = []
predicted2 = []
predicted3 = []

epoch_iterator = tqdm(dataloader, desc="Iteration")
with no_grad():
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to(device) for t in batch)  # GPU or CPU
        generated_outputs = model.generate(input_ids= batch[0], 
                                           attention_mask = batch[1], 
                                           num_return_sequences = num_return_sequences, 
                                           num_beams = 12)
        paras = para_tokenizer.batch_decode(generated_outputs.detach().cpu().numpy(), 
                                                 skip_special_tokens=True)
        predicted1 += paras[0::3]
        predicted2 += paras[1::3]
        predicted3 += paras[2::3]

## Save results to a csv file

In [10]:
import pandas as pd

In [11]:
# Store outputs to disk using in_filename as the original texts 
# and writing outputs to out_filename

# If you want to do other parts of the dataset other than train, 
# set the mode in 'dataset' above to the desired mode and then rerun the paraphrase
# and change these filenames to point to the slice of the data you want to use (dev, test, etc.)
in_filename = 'train.csv'
out_filename = 'train_transfered.csv'

df_para = pd.DataFrame(data={'transfered1' : predicted1, 
                             'transfered2' : predicted2, 
                             'transfered3' : predicted3}) 
df = pd.read_csv(os.path.join(data_dir, paraphrase_task, in_filename), names =['text', 'label'])
df['transfered1'] = df_para['transfered1']
df['transfered2'] = df_para['transfered2']
df['transfered3'] = df_para['transfered3']
df.to_csv(os.path.join(data_dir, prompt_task, out_filename), 
               header=False, index=False)

In [16]:
# Inspect some results
df.head(100)

,text,label,transfered1,transfered2,transfered3
0,i have not read them i have only seen the firs...,0,I have not read them i' th' first movie.,"I have not read them, i' th' first movie i' th","I have not read them, i' th' first movie."
1,"I know they are stupid, and hardly make any se...",0,"I know they are stupid, and hardly make any se...","I know they are stupid, and make no sense at all.","I know they are stupid, and hardly make any se..."
2,put more in what you right,0,Put more in what thou hast righted.,Put more in what thou hath righted.,Put more in what thou right?
3,Give it a miss and get some N.W.A.,0,Give it miss and get some N.W.A.,"Give it miss, and get some N.W.A.",Give it misses and get some N.W.A.
4,"Flash, because that is his only power.",0,"Flash, for that is his only power.","Flash, for that is only his power.","Flash, for that is his power only."
...,...,...,...,...,...
95,Let every eye nogetiate for itself And trust n...,1,Let every eye nogetiate for itself And trust n...,Let every eye nogetiate For itself And trust n...,Let every eye nogetiate for itself And trust n...
96,it just means you have moved on to accept that...,0,It just means you have moved on to accept that...,This just means you have moved on to accept th...,It just means that you have moved on to accept...
97,there's no problem falling in love with your g...,0,There’s no problem falling in love with your g...,There’s no problem in falling in love with you...,There is no problem in falling in love with th...
98,MY CHICKEN I VERY FUNKY HE HAS RED SPOTS AND Y...,0,I VERY FUNKY HE HAS RED SPOTS AND YEL,MY CHICKEN I VERY FUNKY HE HAS RED SPO,My CHICKEN I VERY FUNKY HE HAS RED SPOTS


In [13]:
df.tail()

,text,label,transfered1,transfered2,transfered3
31734,"Why then, God’s soldier be he!",1,"Why then, God’s soldier be he!","Why then, God be a soldier!","Why then, God’s soldier be he."
31735,If he cheated once odds are he will again.,0,"If he cheated once odds are, he will again.",If he cheated once odds are he will again.,"If he cheated once odds are, then he will again."
31736,"In me ’tis villainy, In thee ’t had been good ...",1,"In me ’tis villainy, In thee ’twas good service.","Paraphrase: In me ’tis villainy, In thee ’twas","In me ’tis villainy, In thee ’t had been good ..."
31737,It is perfectly fine. People who are gay shoul...,0,It is perfectly fine. People that are gay shou...,"It is perfectly fine, and those that are gay s...",It is perfectly fine. People who are gay shoul...
31738,"Some never, some sometimes & some all the time...",0,"Some never, some sometimes & some all the time...","Some never, Some sometimes & some all the time...","Some never, some sometimes & some all the time..."


## Now use classifier for Scoring
This may cause GPU memory issues, so it's possible you may have to shutdown the kernel and restart without running the paraphraser first to run this next portion. If doing so, reload the df that was written to disk in several cells above.  

## Load in desired dataset and classifier model
In the cell below, define the dataset you want to work with and the classifier model.

In [14]:
model_config = AutoConfig.from_pretrained(model_args_joint.model_name_or_path, 
                                          cache_dir=model_args_joint.cache_dir)
tokenizer = AutoTokenizer.from_pretrained(model_args_joint.model_name_or_path, 
                                          cache_dir=model_args_joint.cache_dir,
                                          model_max_length = data_args_joint.max_seq_len)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at ../models/cache/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.3.3",
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at ../models/cache/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd

In [15]:
# Load data as expected by joint classifier
tasks = data_args_joint.task.split('+')
train_dataset, idx_to_classes = jld(data_args_joint.data_dir, 
                                             tokenizer, 
                                             model_name=model_args_joint.model_name_or_path, 
                           tasks=tasks, mode="train", n_proc=6000)
dev_dataset, _ = jld(data_args_joint.data_dir, 
                              tokenizer, 
                              model_name=model_args_joint.model_name_or_path, 
                              tasks=tasks, mode="dev", n_proc=6000)

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed_filtered/abstract/config.json'

In [ ]:
label_dims = {task : 1 if len(list(idx_to_classes[task].keys())) == 2 else len(list(idx_to_classes[task].keys())) for task in idx_to_classes}
label_dims

In [ ]:
joint_model = JointSeqClassifier.from_pretrained(os.path.join(output_dir,
                                                              model_args_joint.model_nick, joint_task),
                                           tasks=tasks,
                                           model_args=model_args_joint,
                                           task_if_single=None, 
                                           joint = training_args_joint.train_jointly,
                                           label_dims=label_dims)

trainer = JointTrainer([training_args_joint,model_args_joint, data_args_joint], 
                       joint_model, train_dataset, dev_dataset, idx_to_classes)

## Run classifier on paraphrased and original text

This is currently done with pd DataFrames but could probably be made better by using a batch data loader. 

In [ ]:
import scipy.stats as ss
from tqdm import tqdm
tqdm.pandas()

In [ ]:
tasks

In [ ]:
def pred_paraphrases(row, tasks, cols):
    '''
    Make style predictions on a given df row for a given set of text columns
    and classification tasks. 
    '''
    preds = {}
    for col in cols:
        sentence = row[col]
        out = trainer.predict_for_sentence(sentence, tokenizer)
        for task in tasks:
            pred = float(out[task]['prob'])
            preds[task + '_' + col] = pred
    return preds

def get_best_pred(row, cols, target_val=0.5):
    '''
    Helper funtion for determiningg which paraphrase is 'best' 
    for a given set of paraphrase column style scores and a target value
    that you want the scores to be close to. Currently just outputs the best score
    but could be modified to get best sentence as well.
    '''
    best_diff = 1
    best_val = None
    for col in cols:
        diff = abs(row[col] - target_val)
        if diff < best_diff:
            best_val = row[col]
            best_diff = diff
    return best_val

In [ ]:
# Define columns on which to run the classification
cols_to_use = ['text','transfered1', 'transfered2', 'transfered3']
# Define the names of the columns where the output scores will be stored
cols_preds = [f'pred_{tasks[0]}_orig', f'pred_{tasks[1]}_orig',
              f'pred_{tasks[0]}_transfered1', f'pred_{tasks[1]}_transfered1',
              f'pred_{tasks[0]}_transfered2', f'pred_{tasks[1]}_transfered2',
              f'pred_{tasks[0]}_transfered3', f'pred_{tasks[1]}_transfered3']
# Store results into df
df[cols_preds] = df.progress_apply(lambda x : pred_paraphrases(x, tasks, cols_to_use), 
                                   axis=1, result_type="expand")

In [ ]:
# Store results of style classification:
out_filename = paraphrase_task + '_train_cross_predict_transfers.csv'

df.to_csv(os.path.join(data_dir, prompt_task, out_filename), header=True, index=False)

## Analysing the results of the transfer for style changes

In [ ]:
df.head(100)

In [ ]:
df[f'{prompt_task}_diff1'] =  abs(df[f'pred_{prompt_task}_orig'] - df[f'pred_{prompt_task}_transfered1'])
df[f'{prompt_task}_diff2'] =  abs(df[f'pred_{prompt_task}_orig'] - df[f'pred_{prompt_task}_transfered2'])
df[f'{prompt_task}_diff3'] =  abs(df[f'pred_{prompt_task}_orig'] - df[f'pred_{prompt_task}_transfered3'])

In [ ]:
df['formality_diff_max'] = df.apply(lambda x : np.max([x[f'{prompt_task}_diff1'], 
                                                       x[f'{prompt_task}_diff2'], 
                                                       x[f'{prompt_task}_diff3']]), axis=1)

Let's look at best sytle difference summary stats

In [ ]:
df[f'{prompt_task}_diff_max'].describe()

Let's disaggregate by class label

In [ ]:
df[df['label']==0][f'{prompt_task}_diff_max'].describe()

In [ ]:
df[df['label']==1][f'{prompt_task}_diff_max'].describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()